<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 2 - CSP <br>
<font color=696880 size=4>
    Yalda Shabanzadeh, Ali Salesi

____

# Personal Data

In [ ]:
# Set your student number
student_number = ''
name = ''
last_name = ''

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q1: Cryptarithmetic Puzzle (100 Points)

<font size=4>
Author: Yalda Shabanzadeh
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

## Problem Definition

The Cryptarithmetic Puzzle is a puzzle where the digits of some numbers are replaced with letters to make a mathematical equation. Each letter represents a unique digit from 0 to 9. The puzzle has the following constraints:

- Each letter can be replaced by a digit from 0 to 9, but no digit can be used twice.
- Each letter must be replaced by the same digit throughout the puzzle.
- The leftmost digit of each number cannot be zero.

### Example
Here's an example puzzle:

```
  TWO
+ TWO
------
 FOUR
```

In this puzzle, we need to replace each letter with a digit from 0 to 9 to make the equation true.

## Note
- You can use any arbitrary CSP algorithm to solve this question. But it is recommended to use more complex algorithms so as not to face the time limit.
- Allowed algorithms:
BackTracking, AC3, AC3-LCV, AC3-MRV, AC3-LCV-MRV

> Explain how you consider this problem as a CSP (5 points)


<font color='cyan'>
Your explanation.
</font>


```
Variables: Each letter in the puzzle is a variable, represented as a string.
Domains: The domain of each variable is the set of digits from 0 to 9 that are not already assigned to another variable.
Constraints:
        - All variables must have a value assigned to them.
        - No digit can be assigned to more than one variable.
        - The leftmost digit of each number cannot be zero.
        - The sum of the digits in each column must be consistent with the digit in the corresponding column of the result.
```

## Input
  - The first line is $n$ (count of strings to add up together).
  - In the following $n$ lines, the $i$ th line ($1 \le i \le M$) is $i$ th string.
  - In the next line, $result$, the sum of strings is given.
  - **Note**: your code should be case insensitive to pass the tests (there is no difference between `send` and `SEND`).
 
$$1 \le N \le 50$$

### Sample Input
This sample describes the previous example.
```
2
TWO
TWO
FOUR
```

## Output
for each given charachter as input, print it's corresponding number:
  - **Note**: order is not important.

### Sample Output
The solution to the example above is:
```
F 1
O 4
R 8
T 7
U 6
W 3
```

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. If you tried to use another structure, the total score of that part won't be changed.

### Define a CSP class (50 points)

You can add functions you'll need to this class.

In [22]:
class CSP:
    def __init__(self, variables, domains) -> None:
        self.variables = variables  # variables to be constrained
        self.domains = domains      # domain of each variable
        self.constraints = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint) -> None:
        """
        Add a constraint to the problem
        :param constraint: PuzzleConstraint
        :return: None
        """
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    def consistent(self, variable, assignment) -> bool:
        """
        Check if the value assignment is consistent
        """
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment=None):
        # assignment is complete if every variable is assigned (our base case)
        if assignment is None:
            assignment = {}
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned = [
            v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result = self.backtracking_search(
                    local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None

### Define Constraint (20 points)

In this part, write the satisfaction part of the constraint of this problem.
You can add any argument you want to `__init__` function.

In [21]:
class PuzzleConstraint:
    def __init__(self,
                 variables,
                 added_up,
                 result) -> None:

        self.variables: list[str] = variables
        self.added_up = added_up
        self.expected_result: str = result

    # Convert a word into a number based on the sum of the values given on the dictionary
    # for each letter in the word.

    def to_number(self, word: str, letter_values):
        sum: int = 0
        base: int = 1

        # Sum the values of the letters in the word.
        for letra in reversed(word):
            sum += letter_values[letra] * base
            base *= 10

        # Return the sum of the values of the letters in the word.
        return sum

    # Check if condition is satisfied to return True or False.

    def satisfied(self, assignment):

        # Check if all letters have a different value.
        if len(set(assignment.values())) != len(assignment):
            return False

        # Check if the first letter of the word is not 0.
        for word in [*self.added_up, self.expected_result]:
            if word[0] in assignment and assignment[word[0]] == 0:
                return False

        if len(assignment) < len(self.variables):
            return True
        sum: int = 0

        # Sum the values of the letters in the words.
        for word in self.added_up:
            number: int = self.to_number(word, assignment)
            sum += number

        # Return True if the sum of the values of the letters in the words is
        # equal to the value of the letters in the result word.

        return sum == self.to_number(self.expected_result,
                                     assignment)

### Build CSP and solve it! (20 points)

complete these functions for testing.

In [23]:
def get_problem(added_up, result):
    """
    Define the CSP problem
    
    :param added_up: List[str]
    :param result: str
    :return: return the CSP
    """
    for i in range(len(added_up)):
        added_up[i] = added_up[i].upper()

    result = result.upper()
    variables: set[str] = set()

    # Add the letters of the words to the list of variables.
    for p in added_up:
        for l in p:
            variables.add(l)

    # Add the letters of the result word to the list of variables.
    for l in result:
        variables.add(l.upper())

    # Create a list of all the possible values for each letter.
    variables_list: list[str] = list(variables)
    variables_list.sort()

    domains: dict[str, list[int]] = {var: list(range(10))
                                     for var in variables_list}

    csp: CSP = CSP(variables_list, domains)

    # Add the constraint to the CSP.
    csp.add_constraint(PuzzleConstraint(variables_list, added_up, result))
    return csp

In [24]:
def backtracking_search(csp):
    """
    solve CSP

    :param csp: CSP
    :return: return the founded solution
    """
    # Solve the CSP.
    sol = csp.backtracking_search()
    res = ''
    for k, v in sol.items():
        res += f'{k} {v}\n'

    # Return the solution.
    return res

## Test

**Do not change the following cell.**

**Note**: 5 tests will be added after deadline (5 points).

In [ ]:
import helper.test_q2 as q2
import time

TIME_LIMIT = 3

tests = q2.get_all_tests(prefix='q2_')
tests_passed = 0
for test in tests:
    n, n_next_lines, result = q2.scan_test_input(test)
    n_next_lines = ['two', 'two']
    result = 'four'
    csp = get_problem(n_next_lines, result)
    start_time = time.time()
    result = backtracking_search(csp)
    print(result)
    total_time = time.time() - start_time
    if q2.is_result_valid(test, result) and total_time < TIME_LIMIT:
        tests_passed += 1
    else:
        print(f'test {test} failed. time elapsed= {total_time}')
print(f'Score = {tests_passed / len(tests) * 100}%')